In [343]:
from openai import OpenAI
from secret import OPENAI_API_KEY
from PIL import Image
import base64
import os
import json
import re
import pandas as pd
import io
from tabulate import tabulate

In [344]:
client = OpenAI(api_key=OPENAI_API_KEY)
model = "gpt-4o"

In [345]:
def encode_image(image_path, max_size=(512, 512), quality=80):
    image = Image.open(image_path)

    # Redimensionner l'image
    image.thumbnail(max_size)

    # Convertir en bytes avec compression
    buffer = io.BytesIO()
    image.save(buffer, format="JPEG", quality=quality)

    # Encoder en Base64
    encoded_string = base64.b64encode(buffer.getvalue()).decode("utf-8")

    return encoded_string

In [346]:
def old_encode_image(image_path):
     try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")

     except Exception as e:

        print(f"Erreur lors de l'encodage de l'image {image_path} : {e}")
        return None

In [347]:
def send_chat_request(message):
    try:
        response = client.chat.completions.create(
            model=model, messages=message
        )

        result = response.choices[0].message.content.strip()
        result = extract_json(result)

        tokens = response.usage.total_tokens

        return result, tokens


    except Exception as e:
        print(f"Erreur OpenAI : {e}")
        return -1, None

In [348]:
def chat_get_key_words(image_paths):

    # Liste pour chaque image et chaque texte associé
    content_list = []
    for image_path in image_paths:
        base64_image = encode_image(image_path)
        image_name = os.path.basename(image_path)
        content_list.append({
            "type": "text",
            "text": f"""Décris moi l'image avec 5 mots-clés.Retourne le résultat au format JSON : {{ {image_name} : [mot-clé1, mot-clé2, mot-clé3, mot-clé4, mot-clé5] }} """
        })
        content_list.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
        })

    messages = [
        {
            "role": "user",
            "content": content_list
        }
    ]

    return send_chat_request(messages)


In [349]:
def extract_json(response_text):
    """
    Extrait la portion JSON (délimitée par {}) de la réponse textuelle pour seulement avoir le dictionnaire et non le texte généré par l'ia.
    """
    match = re.search(r'\{.*\}', response_text, re.DOTALL)
    if match:
        json_str = match.group()
        try:
            return json.loads(json_str)
        except Exception as e:
            print(f"Erreur lors du chargement du JSON : {e}")
            return None
    else:
        print("Aucun JSON trouvé dans la réponse.")
        return None

In [350]:
def chat_get_categories(keywords_output):
    """
    Utilise les mots-clés extraits pour regrouper les images similaires en catégories.
    Les images sont identifiées par leur ordre dans la liste.
    """

    # Préparation d'un prompt détaillé incluant le résultat des mots-clés et l'ordre des images
    prompt = f"""Voici les listes de mots-clés obtenues pour chaque image (dans l'ordre) : {keywords_output}
    Regroupe les images similaires dans des catégories. Une catégorie est décrite par un seul mot-clé. Une image ne peut appartenir qu'à une seule catégorie. Retourne le résultat au format JSON : {{ "categorie1": [ "name", "name" ], "categorie2": [ "name", "name" ],...}}"""

    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]

    return send_chat_request(messages)

In [351]:
directory = "photos_victor"
allowed_extensions = {".jpg", ".jpeg", ".png"}
image_paths = [
    os.path.join(directory, filename)
    for filename in os.listdir(directory)
    if os.path.splitext(filename)[1].lower() in allowed_extensions
]

## Ajout des données au DataFrame

In [352]:
def create_df(image_paths):
    image_list = []
    for path in image_paths:
        image = Image.open(path)
        image_name = os.path.basename(path)
        exifdata = image._getexif()
        date_time, localisation = None, None
        if exifdata:
            for tag_id, value in exifdata.items():
                tag = Image.ExifTags.TAGS.get(tag_id, tag_id)
                if tag == "DateTime":
                    date_time = value
                elif tag == "GPSInfo":
                    localisation = value

            image_list.append((image_name, path, date_time, localisation))

        else:
            print("Aucune donnée EXIF trouvée.")

    df = pd.DataFrame(image_list, columns=["image_name", "path", "date_time", "localisation"])

    return df

In [353]:
def add_keywords_to_df(image_data, keywords_output):
    if keywords_output:
        image_data.loc[image_data["image_name"].isin(keywords_output.keys()), "keywords"] = image_data["image_name"].map(keywords_output)
    else :
        print("Aucun mot clé fourni ! ")
    return image_data


In [354]:
def add_categories_to_df(image_data, categories_output):
    if categories_output:
        #Inversion du dict : on associe une categorie a chaque image
        image_to_categories = {img: cat for cat, images in categories_output.items() for img in images}

        image_data.loc[image_data["image_name"].isin(image_to_categories.keys()), "categories"] = image_data["image_name"].map(image_to_categories)
    else:
        print("Aucune catégorisation trouvée !")

    return image_data

In [355]:
def pipeline_keywords(image_paths, limit_size=10):
    image_data = create_df(image_paths)
    stop = False
    total_keywords_tokens = 0
    for i in range(0, len(image_paths), limit_size):
        interval = [i, min(i + limit_size, len(image_paths))]
        subset_image_paths = image_paths[interval[0]:interval[1]]
        print(f"Image paths : {subset_image_paths}")

        keywords_output, keywords_tokens = chat_get_key_words(subset_image_paths)
        total_keywords_tokens += keywords_tokens

        image_data = add_keywords_to_df(image_data, keywords_output)

        print(f"Total tokens : {total_keywords_tokens}")

        if stop :
            break

    return image_data, total_keywords_tokens

In [356]:
image_data, keywords_tokens = pipeline_keywords(image_paths, 10)

Image paths : ['photos_victor\\IMG_20241228_124125_1.jpg', 'photos_victor\\IMG_20241228_124128.jpg', 'photos_victor\\IMG_20241228_124135.jpg', 'photos_victor\\IMG_20241228_124137.jpg', 'photos_victor\\IMG_20241228_124140.jpg', 'photos_victor\\IMG_20241228_124648.jpg', 'photos_victor\\IMG_20241228_132157.jpg', 'photos_victor\\IMG_20241228_141921.jpg', 'photos_victor\\IMG_20241229_123811.jpg', 'photos_victor\\IMG_20241229_123828.jpg']
Total tokens : 3272
Image paths : ['photos_victor\\IMG_20241229_123844.jpg', 'photos_victor\\IMG_20241229_124813.jpg', 'photos_victor\\IMG_20241229_124828.jpg', 'photos_victor\\IMG_20241229_125737.jpg', 'photos_victor\\IMG_20241229_125929.jpg', 'photos_victor\\IMG_20241229_130006.jpg', 'photos_victor\\IMG_20241229_130244.jpg', 'photos_victor\\IMG_20241229_130249.jpg', 'photos_victor\\IMG_20241229_130430.jpg', 'photos_victor\\IMG_20241229_130435.jpg']
Total tokens : 6527
Image paths : ['photos_victor\\IMG_20241229_130635.jpg', 'photos_victor\\IMG_20241229_13

In [357]:
print(tabulate(image_data, headers="keys", tablefmt="psql"))

+----+--------------------------------+----------------------------------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------+
|    | image_name                     | path                                         | date_time           | localisation                                                                                                                                                         | keywords                                                            |
|----+--------------------------------+----------------------------------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------

In [358]:
copy_image_data = image_data.copy()

In [359]:
def pipeline_categories(image_data, limit_size=200):
    keywords = image_data.set_index("image_name")["keywords"].to_dict()
    total_categories_tokens = 0

    for i in range(0, len(keywords), limit_size):
        interval = [i, min(i + limit_size, len(image_data))]
        subset_keys = list(keywords.keys())[interval[0]:interval[1]]

        subset_keywords = {key: keywords[key] for key in subset_keys}

        categories_output, categories_tokens = chat_get_categories(subset_keywords)
        total_categories_tokens += categories_tokens

        image_data = add_categories_to_df(image_data, categories_output)

    return image_data, total_categories_tokens

In [360]:
copy_image_data, categories_tokens = pipeline_categories(copy_image_data)

In [361]:
print(tabulate(copy_image_data, headers="keys", tablefmt="psql"))

+----+--------------------------------+----------------------------------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------+--------------+
|    | image_name                     | path                                         | date_time           | localisation                                                                                                                                                         | keywords                                                            | categories   |
|----+--------------------------------+----------------------------------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+---